In [66]:
import numpy as np

# 동형암호 시뮬레이터 모듈
from cipher import *
from scheme import *
from algorithms import Algorithms 
from stats import Statistics

동형암호 (글로벌) 파라미터 설정

In [67]:
myring = Ring(seed=1234)
parms = Parameters(logp = 30, logq = 150, logn = 12)
nslots = 2**parms.logn
context = Context(parms, myring)

sk = context.generate_secret_key()
print(context.params)

keys = {"mult":context.generate_mult_key(),
        "rot":{'1':'hi1',
               '2':'hi2',
               '4':'hi4',
               '8':'hi8'}}

# 암호화, 복호화, 연산 agent
ev = Evaluator(keys, context) 
encoder = Encoder(context)
encryptor = Encryptor(context)
decryptor = Decryptor(sk)

 logp: 30
 logq: 150 
 logn: 12


In [68]:
# 통계 연산 모듈
algo = Algorithms(ev, encoder)
st = Statistics(ev, encoder)
check = Checker()

### variance

#### 일반

$$
var = \frac{\Sigma(x_i - (\bar{x}))^2}{n}
$$

In [71]:
arr = np.array([1,2,3,4,5,6,7,8])

mean = np.mean(arr)
var = np.sum((arr - mean)**2)/len(arr)
print("일반 알고리즘,", var)
print("numpy' result,", np.var(arr))

일반 알고리즘, 5.25
numpy' result, 5.25


#### 동형암호 버전

In [70]:
ctxt = encryptor.encrypt(arr)
var = st.variance(ctxt)

check.close(var._arr[0], np.var(arr))

Absolute error within: 1.0E-06
Relative error within: 1.0E-04


## 소스코드 
https://github.com/Deep-In-Sight/FHE_simulator/blob/9fb4ce2e0ea513181b7b29626de5120947e19bea/stats.py#L18

```python
def variance(self, ctxt):
    ev = self.evaluator
    algo = self.algorithms

    n = algo.encode_repeat(ctxt._n_elements)
    mean = self.mean(ctxt, partial=True, duplicate=True)
    self.evaluator.rescale_next(mean)
    mean = algo.put_mask(mean, np.arange(ctxt._n_elements))
    self.evaluator.rescale_next(mean)
    sub = ev.sub(ctxt, mean)
    squared = ev.mult(sub, sub, inplace=False) # Or, square()
    self.evaluator.rescale_next(squared)

    summed_sq = algo.sum_reduce(squared, partial=True, duplicate=False)
    return ev.div_by_plain(summed_sq, n)


def mean(self, ctxt, partial=True, duplicate=True):
    ev = self.evaluator
    algo = self.algorithms

    n = algo.encode_repeat(ctxt._n_elements)
    summed = algo.sum_reduce(ctxt, partial=partial, duplicate=duplicate)
    return ev.div_by_plain(summed, n)


def sum_reduce(self,
                ctxt:CiphertextStat, 
                partial=False, 
                duplicate=False): 
    if not partial and duplicate:
        raise ValueError("Partial = False, duplicate = True not allowed.")
    ev = self.evaluator

    if partial:
        n = ctxt._n_elements
    else:
        n = ctxt.nslots
    log2n = np.log2(n).astype(int)

    # keep the original ctxt intact
    ctxt_ = ev.copy(ctxt)
    if duplicate:
        # shifted copy
        rot_copy = ev.copy(ctxt)
        ev.lrot(rot_copy, -ctxt_._n_elements)

        ev.add(ctxt_, rot_copy, inplace=True)
    for i in range(log2n):
        tmp = ev.copy(ctxt_)
        ev.lrot(tmp, 2**i, inplace=True)
        ev.add(ctxt_, tmp, inplace=True)
    return ctxt_
```